## Modeling!

I've built my fair share of different card game models for programming courses and puzzles like these so I knew I wanna make the hand of cards into a class that knows its own value and can be sorted with other Hands.

I first build an [Enum](https://docs.python.org/3/library/enum.html) for the values of different hands. This way, we can deal with named hands and only refer to their ranking when needed for sorting. Better hand, higher value.

Next, I created a dictionary to map characters into their numeric values. This is also used for sorting only.

In [1]:
from enum import Enum


class HandValue(Enum):
    HIGH_CARD = 1
    ONE_PAIR = 2
    TWO_PAIRS = 3
    THREE_OF_A_KIND = 4
    FULL_HOUSE = 5
    FOUR_OF_A_KIND = 6
    FIVE_OF_A_KIND = 7

NUMERIC_VALUES = {
    '2': 2,
    '3': 3,
    '4': 4,
    '5': 5,
    '6': 6,
    '7': 7, 
    '8': 8,
    '9': 9,
    'T': 10,
    'J': 11,
    'Q': 12,
    'K': 13,
    'A': 14
}

For the `Hand` class, I store the cards themselves, a [set](https://docs.python.org/3/library/stdtypes.html#set) of them and a card count [Counter](https://docs.python.org/3/library/collections.html#collections.Counter).

I then built a function for each hand type (except the `HIGH_CARD` which is default if nothing else is met.

To get the value of a hand, I go down from the best hand to worst, and check until a match is found.

To compare two hands, they are either sorted by their full hand value and if that is the same, card by card by their numeric value.

In [2]:
from collections import Counter


class Hand:
    
    def __init__(self, cards):
        self.cards = list(cards)
        self.card_set = set(cards)
        self.card_counts = Counter(cards)
        self.value = self.get_value()
        self.numeric_values = NUMERIC_VALUES
        
    def is_five_of_a_kind(self):
        return len(self.card_set) == 1
    
    def is_four_of_a_kind(self):
        if len(self.card_set) != 2:
            return False
        return any(c for c in self.card_counts.values() if c == 4)
    
    def is_full_house(self):
        has_triplet = any(c for c in self.card_counts.values() if c == 3)
        has_pair = any(c for c in self.card_counts.values() if c == 2)
        return has_triplet and has_pair
    
    def is_three_of_a_kind(self):
        return any(c for c in self.card_counts.values() if c == 3)
    
    def is_two_pairs(self):
        pairs = [c for c in self.card_counts.values() if c == 2]
        return len(pairs) == 2
    
    def is_one_pair(self):
        pairs = [c for c in self.card_counts.values() if c == 2]
        return len(pairs) == 1
        
    def get_value(self):
        if self.is_five_of_a_kind():
            return HandValue.FIVE_OF_A_KIND
        if self.is_four_of_a_kind():
            return HandValue.FOUR_OF_A_KIND
        if self.is_full_house():
            return HandValue.FULL_HOUSE
        if self.is_three_of_a_kind():
            return HandValue.THREE_OF_A_KIND
        if self.is_two_pairs():
            return HandValue.TWO_PAIRS
        if self.is_one_pair():
            return HandValue.ONE_PAIR
        
        return HandValue.HIGH_CARD
    
    def __lt__(self, other):
        if self.value.value != other.value.value:
            return self.value.value < other.value.value
        else:
            for c1, c2 in zip(self.cards, other.cards):
                c1_value = self.numeric_values[c1]
                c2_value = self.numeric_values[c2]
                if c1_value == c2_value:
                    continue
                return c1_value < c2_value
    
    def __repr__(self):
        return f'<Hand: {"".join(self.cards)} {self.value}>'

## Read input

Today's input is simple. Each line has the cards and the bid separated by space. I convert the cards to a `Hand` and bid to an `int`.

In [3]:
from utils import read_input


def transformer(line):
    hand, bid = line.split(' ')
    hand = Hand(hand)
    bid = int(bid)
    return hand, bid

hands = read_input(7, transformer)

To solve the first part, I loop over all the hands in sorted order and multiply their rank with their bid.

In [4]:
part_1 = 0
for i, (hand, bid) in enumerate(sorted(hands)):
    part_1 += (i+1) * bid
print(part_1)
assert part_1 == 253205868

253205868


## Part 2

>To make things a little more interesting, the Elf introduces one additional rule. Now, J cards are jokers - wildcards that can act like whatever card would make the hand the strongest type possible.
>
>To balance this, J cards are now the weakest individual cards, weaker even than 2. The other cards stay in the same order: A, K, Q, T, 9, 8, 7, 6, 5, 4, 3, 2, J.
>
>J cards can pretend to be whatever card is best for the purpose of determining hand type; for example, QJJQ2 is now considered four of a kind. However, for the purpose of breaking ties between two hands of the same type, J is always treated as J, not the card it's pretending to be: JKKK2 is weaker than QQQQ2 because J is weaker than Q.
>
>Now, the above example goes very differently:
>```
32T3K 765
T55J5 684
KK677 28
KTJJT 220
QQQJA 483
>```
>- 32T3K is still the only one pair; it doesn't contain any jokers, so its strength doesn't increase.
>- KK677 is now the only two pair, making it the second-weakest hand.
>- T55J5, KTJJT, and QQQJA are now all four of a kind! T55J5 gets rank 3, QQQJA gets rank 4, and KTJJT gets rank 5.
>
>With the new joker rule, the total winnings in this example are 5905.
>
>Using the new joker rule, find the rank of every hand in your set. What are the new total winnings>?

For this second part, the only changes needed to make are in the modeling.

First, we drop the value of 'J' cards to zero for sorting.

Second, thanks to inheritance, we only need to rewrite the methods that are changed by the addition of Joker cards. Notably, two pairs does not change because it can never happen with a Joker as that Joker would turn it into a three of a kind.

Setting up, finding hand value and making comparisons all remain mostly unaffected. We add a `self.joker` attribute for the Joker hand and use `JOKER_NUMERIC_VALUES` for comparison instead of `NUMERIC_VALUES`.

In [5]:
from collections import Counter

JOKER_NUMERIC_VALUES = NUMERIC_VALUES.copy()
JOKER_NUMERIC_VALUES['J'] = 0


class JokerHand(Hand):
    
    def __init__(self, cards):
        self.joker = 'J' in cards
        super().__init__(cards)
        self.numeric_values = JOKER_NUMERIC_VALUES
        
    def is_five_of_a_kind(self):
        if not self.joker:
            return len(self.card_set) == 1
        return len(self.card_set) <= 2 
    
    def is_four_of_a_kind(self):
        if not self.joker:
            if len(self.card_set) != 2:
                return False
            return any(c for c in self.card_counts.values() if c == 4)
        return any(count for card, count in self.card_counts.items() if count == 4 - self.cards.count('J') and card != 'J')

    def is_full_house(self):
        has_triplet = any(c for c in self.card_counts.values() if c == 3)
        has_pair = any(c for c in self.card_counts.values() if c == 2)
        full_house = has_triplet and has_pair
        if full_house:
            return True
        elif not full_house and not self.joker:
            return False
        else: # Not a natural full house but has joker so it might still be
            pairs = [card for card, count in self.card_counts.items() if count == 2]
            return len(pairs) == 2
            
    def is_three_of_a_kind(self):
        return any(c for c in self.card_counts.values() if c == 3 - self.cards.count('J'))
    
    def is_one_pair(self):
        if self.joker:
            return True
        pairs = [c for c in self.card_counts.values() if c == 2]
        return len(pairs) == 1

When reading input, we convert hands into `JokerHand`s instead and otherwise it's the same.

In [6]:
def transformer(line):
    hand, bid = line.split(' ')
    hand = JokerHand(hand)
    bid = int(bid)
    return hand, bid

jokerhands = read_input(7, transformer)

The actual solution is exactly the same at this point than part 1.

In [7]:
part_2 = 0
for i, (hand, bid) in enumerate(sorted(jokerhands)):
    part_2 += (i+1) * bid
print(part_2)
assert part_2 == 253907829

253907829


## Two stars!

7 days, 14 stars. Woohoo! 36 to go.